In [39]:
import pyprind
import pandas as pd
import os

"""basepath = 'aclImdb'

labels = {'pos' : 1, 'neg' : 0}
pbar = pyprind.ProgBar(50000)
df = pd.DataFrame()"""

"basepath = 'aclImdb'\n\nlabels = {'pos' : 1, 'neg' : 0}\npbar = pyprind.ProgBar(50000)\ndf = pd.DataFrame()"

In [40]:
"""for s in ('test', 'train'):
    for l in ('pos', 'neg'):
        path = os.path.join(basepath, s, l)
        for file in sorted(os.listdir(path)):
            with open(os.path.join(path, file), 'r', encoding='utf-8') as infile:
                txt = infile.read()
            df = df.append([[txt, labels[l]]], ignore_index=True)
            pbar.update()

df.columns = ['review', 'sentiment']"""

"for s in ('test', 'train'):\n    for l in ('pos', 'neg'):\n        path = os.path.join(basepath, s, l)\n        for file in sorted(os.listdir(path)):\n            with open(os.path.join(path, file), 'r', encoding='utf-8') as infile:\n                txt = infile.read()\n            df = df.append([[txt, labels[l]]], ignore_index=True)\n            pbar.update()\n\ndf.columns = ['review', 'sentiment']"

In [41]:
import numpy as np
np.random.seed(0)
"""df = df.reindex(np.random.permutation(df.index))
df.to_csv('movie_data.csv', index=False, encoding='utf-8')"""

"df = df.reindex(np.random.permutation(df.index))\ndf.to_csv('movie_data.csv', index=False, encoding='utf-8')"

In [42]:
df = pd.read_csv('movie_data.csv', encoding='utf-8')
df.shape

(50000, 2)

In [43]:
from sklearn.feature_extraction.text import CountVectorizer
count = CountVectorizer()#создаем 1-граммы, чтобы создать 2-граммы передаем ngram_range = (2, 2)
docs = np.array(['The sun is shining',
                 'The weather is sweet',
                 'The sun is shining, the weather is sweet, and one and one is two'])
bag = count.fit_transform(docs)
count.vocabulary_#глосарий - слово и индекс

{'the': 6,
 'sun': 4,
 'is': 1,
 'shining': 3,
 'weather': 8,
 'sweet': 5,
 'and': 0,
 'one': 2,
 'two': 7}

In [44]:
count = CountVectorizer(ngram_range = (2, 2))
docs = np.array(['The sun is shining',
                 'The weather is sweet',
                 'The sun is shining, the weather is sweet, and one and one is two'])
bag = count.fit_transform(docs)#fit - создаем глоссарий, transform - трансформируем предложения в вектора признаков
count.vocabulary_

{'the sun': 9,
 'sun is': 7,
 'is shining': 1,
 'the weather': 10,
 'weather is': 11,
 'is sweet': 2,
 'shining the': 6,
 'sweet and': 8,
 'and one': 0,
 'one and': 4,
 'one is': 5,
 'is two': 3}

In [45]:
#модель суммирования слов - вектор признаков для каждого документа
bag.toarray()#вектора признаков, сырые частоты термов 
#tf(t, d) - сколько раз терм t встречается в документе d

array([[0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0],
       [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1],
       [2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]], dtype=int64)

In [46]:
#мера: "частота терма - обратная частота документа" позволяет найти часто встречающиеся слова
#которые не имеют смысла tf-idf (t, d) = tf(t, d) * idf(t, d)
#idf(t, d) = log(nd/(1 + df(t, d))) nd - общее количество документов
#df(t, d) - количество документов d содержащих терм t

In [47]:
from sklearn.feature_extraction.text import TfidfTransformer
tfidf = TfidfTransformer(use_idf=True,
                         norm='l2',#нормализуем вектор термов деля на L2 норму
                         smooth_idf=True)
np.set_printoptions(precision=2)
print(tfidf.fit_transform(count.fit_transform(docs)).toarray())

[[0.   0.58 0.   0.   0.   0.   0.   0.58 0.   0.58 0.   0.  ]
 [0.   0.   0.58 0.   0.   0.   0.   0.   0.   0.   0.58 0.58]
 [0.57 0.22 0.22 0.28 0.28 0.28 0.28 0.22 0.28 0.22 0.22 0.22]]


In [48]:
#в sklearn формулы отличаются, а именно:
#idf(t, d) = log(1 + nd/(1 + df(t, d)))
#tf-idf (t, d) = tf(t, d) * (idf(t, d) + 1) из-за того что smooth_idf=True чтобы  = 0 для термов встречающихся во всех документах


In [49]:
df.loc[0, 'review'][-50:] #данные необходимо очитстить

'is seven.<br /><br />Title (Brazil): Not Available'

In [50]:
import re
def preprocessor(text):
    text = re.sub('<[^>]*>', '', text) #удаление html разметки
    emoticons = re.findall('(?::|;|=)(?:-)?(?:-\)|\(|D|Р)',
                           text)#находми все эмотиконы
    text = (re.sub('[\W]+', ' ', text.lower()) + #находим и удалям все несловарные символы
            ' '.join(emoticons).replace('-', ''))#возвращаем эмотиконы
    return text
    

In [51]:
preprocessor(df.loc[0, 'review'][-50:])

'is seven title brazil not available'

In [52]:
df['review'] = df['review'].apply(preprocessor)

In [53]:
def tokenizer(text):#один из вариантов разбиения на лексемы
    return(text.split())
tokenizer('runners like running and thus they run')

['runners', 'like', 'running', 'and', 'thus', 'they', 'run']

In [54]:
#аналог - стемминг - преобразование слова в его корневую форму
from nltk.stem.porter import PorterStemmer
porter = PorterStemmer()
def tokenizer_porter(text):
    return [porter.stem(word) for word in text.split()]
tokenizer_porter('runners like running and thus they run')

['runner', 'like', 'run', 'and', 'thu', 'they', 'run']

In [55]:
#удаление стоп-слов - слова, которые распространены в тексте и вероятно не несут в себе полезной информации
import nltk
from nltk.corpus import stopwords

stop = stopwords.words('english')
[w for w in tokenizer_porter('runners like running and thus they run')[-10:]
 if w not in stop]

['runner', 'like', 'run', 'thu', 'run']

In [56]:
X_train = df.loc[:25000, 'review'].values
y_train = df.loc[:25000, 'sentiment'].values
X_test = df.loc[25000:, 'review'].values
y_test = df.loc[25000:, 'sentiment'].values

In [57]:
"""from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(strip_accents=None,
                        lowercase=False,
                        preprocessor=None)

param_grid = [{'vect__ngram_range': [(1,1)],
               'vect__stop_words': [stop, None],
               'vect__tokenizer': [tokenizer,
                                  tokenizer_porter],
               'clf__penalty': ['ll', 'l2'],
               'clf__C': [1.0, 10.0, 100.0]},
              {'vect__ngram_range': [(1,1)],
               'vect__stop_words': [stop, None],
               'vect__tokenizer': [tokenizer,
                                    tokenizer_porter],
               'vect__use_idf': [False],
               'vect__norm' : [None],
               'clf__penalty': ['11', '12'],
               'clf__C': [1.0, 10.0, 100.0]}]

lr_tfidf = Pipeline([('vect', tfidf),
                     ('clf',
                      LogisticRegression(random_state=0,
                                         solver='liblinear'))])
gs_lr_tfidf = GridSearchCV(lr_tfidf, param_grid,
                           scoring='accuracy',
                           cv=5, verbose=2,
                           n_jobs=-1)
gs_lr_tfidf.fit(X_train, y_train)"""

"from sklearn.model_selection import GridSearchCV\nfrom sklearn.pipeline import Pipeline\nfrom sklearn.linear_model import LogisticRegression\nfrom sklearn.feature_extraction.text import TfidfVectorizer\n\ntfidf = TfidfVectorizer(strip_accents=None,\n                        lowercase=False,\n                        preprocessor=None)\n\nparam_grid = [{'vect__ngram_range': [(1,1)],\n               'vect__stop_words': [stop, None],\n               'vect__tokenizer': [tokenizer,\n                                  tokenizer_porter],\n               'clf__penalty': ['ll', 'l2'],\n               'clf__C': [1.0, 10.0, 100.0]},\n              {'vect__ngram_range': [(1,1)],\n               'vect__stop_words': [stop, None],\n               'vect__tokenizer': [tokenizer,\n                                    tokenizer_porter],\n               'vect__use_idf': [False],\n               'vect__norm' : [None],\n               'clf__penalty': ['11', '12'],\n               'clf__C': [1.0, 10.0, 100.0]}]

In [58]:
stop = stopwords.words('english')
def tokenizer_global(text):
    text = re.sub('<[^>]*>', '', text) 
    emoticons = re.findall('(?::|;|=)(?:-)?(?:-\)|\(|D|Р)',
                           text)
    text = (re.sub('[\W]+', ' ', text.lower()) + 
            ' '.join(emoticons).replace('-', ''))
    tokenized = [w for w in text.split() if w not in stop]
    return tokenized

def stream_docs(path):
    with open(path, 'r', encoding='utf-8') as csv:
        next(csv)
        for line in csv:
            text, label = line[:-3], int(line[-2])
            yield text, label

In [59]:
next(stream_docs(path='movie_data.csv'))

('"In 1974, the teenager Martha Moxley (Maggie Grace) moves to the high-class area of Belle Haven, Greenwich, Connecticut. On the Mischief Night, eve of Halloween, she was murdered in the backyard of her house and her murder remained unsolved. Twenty-two years later, the writer Mark Fuhrman (Christopher Meloni), who is a former LA detective that has fallen in disgrace for perjury in O.J. Simpson trial and moved to Idaho, decides to investigate the case with his partner Stephen Weeks (Andrew Mitchell) with the purpose of writing a book. The locals squirm and do not welcome them, but with the support of the retired detective Steve Carroll (Robert Forster) that was in charge of the investigation in the 70\'s, they discover the criminal and a net of power and money to cover the murder.<br /><br />""Murder in Greenwich"" is a good TV movie, with the true story of a murder of a fifteen years old girl that was committed by a wealthy teenager whose mother was a Kennedy. The powerful and rich f

In [60]:
def get_minibatch(doc_stream, size):
    docs, y = [], []
    try:
        for _ in range(size):
            text, label = next(doc_stream)
            docs.append(text)
            y.append(label)
    except StopIteration:
        return None, None
    return docs, y

In [61]:
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.linear_model import SGDClassifier
vect = HashingVectorizer(decode_error='ignore',
                         n_features=2**21,
                         preprocessor=None,
                         tokenizer=tokenizer)
clf = SGDClassifier(loss='log', random_state=1)
doc_stream = stream_docs(path='movie_data.csv')

In [62]:
import pyprind
pbar = pyprind.ProgBar(45)
classes = np.array([0, 1])
for _ in range(45):
    X_train, y_train = get_minibatch(doc_stream, size=1000)
    if not X_train:
        break
    X_train = vect.transform(X_train)
    clf.partial_fit(X_train, y_train, classes=classes)
    pbar.update()

c:\Users\VIKTOR\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\linear_model\_stochastic_gradient.py:163: FutureWarning: The loss 'log' was deprecated in v1.1 and will be removed in version 1.3. Use `loss='log_loss'` which is equivalent.
  warnings.warn(
0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:03


In [63]:
X_test, y_test = get_minibatch(doc_stream, size=5000)
X_test = vect.transform(X_test)
print(clf.score(X_test, y_test))

0.8072


In [64]:
clf.partial_fit(X_test, y_test)

SGDClassifier(loss='log', random_state=1)

In [65]:
#Тематическое моделирование - назначение тем непомеченным документам
#один из приемов - латентное размещение дирихле (LDA)
df = pd.read_csv('movie_data.csv', encoding='utf-8')

In [66]:
count = CountVectorizer(stop_words='english',
                        max_df=.1,# max частота слов подлежащая расмотрению
                        max_features=5000)
X = count.fit_transform(df['review'].values)

In [67]:
X

<50000x5000 sparse matrix of type '<class 'numpy.int64'>'
	with 2780504 stored elements in Compressed Sparse Row format>